In [1]:
import cv2 as cv
from glob import glob
import os
import numpy as np
from poincare import calculate_singularities
from segmentation import create_segmented_and_variance_images
from normalization import normalize
from gabor_filter import gabor_filter
from frequency import ridge_freq
import orientation
from crossing_number import calculate_minutiaes
from tqdm import tqdm
from skeletonize import skeletonize
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
import random


In [2]:
def processing(input_img):
    # normalization -> orientation -> frequency -> mask -> filtering
    block_size = 16

    # normalization - removes the effects of sensor noise and finger pressure differences.
    normalized_img = normalize(input_img.copy(), float(100), float(100))

    # ROI and normalisation
    (segmented_img, normim, mask) = create_segmented_and_variance_images(
        normalized_img, block_size, 0.2
    )

    # orientations
    angles = orientation.calculate_angles(normalized_img, W=block_size, smoth=False)
    orientation_img = orientation.visualize_angles(
        segmented_img, mask, angles, W=block_size
    )

    # find the overall frequency of ridges in Wavelet Domain
    freq = ridge_freq(
        normim,
        mask,
        angles,
        block_size,
        kernel_size=5,
        minWaveLength=5,
        maxWaveLength=15,
    )

    # create gabor filter and do the actual filtering
    gabor_img = gabor_filter(normim, angles, freq)

    # thinning or skeletonize
    thin_image = skeletonize(gabor_img)

    # minutias
    minutiae, minutiae_list = calculate_minutiaes(thin_image)

    # singularities
    singularities_img, singularities_list = calculate_singularities(thin_image, angles, 1, block_size, mask)

    # Combine minutiae_list and singularities_list into features
    features = {
        "minutiae_list": minutiae_list,
        "singularities_list": singularities_list,
    }

    # visualize pipeline stage by stage
    output_imgs = [
        input_img,
        normalized_img,
        segmented_img,
        orientation_img,
        gabor_img,
        thin_image,
        minutiae,
        singularities_img,
    ]

    for i in range(len(output_imgs)):
        if len(output_imgs[i].shape) == 2:
            output_imgs[i] = cv.cvtColor(output_imgs[i], cv.COLOR_GRAY2RGB)
    results = np.concatenate(
        [np.concatenate(output_imgs[:4], 1), np.concatenate(output_imgs[4:], 1)]
    ).astype(np.uint8)

    return results, features, minutiae


In [2]:
# open images
img_dir = "./input/*"
output_dir = "./output/"
# Define the output directory for minutiae images
minutiae_output_dir = "./minutiae/"

def open_images(directory):
    images_paths = glob(directory)
    return [(img_path, cv.imread(img_path, 0)) for img_path in images_paths]


images = open_images(img_dir)

def extract_label(filename):
    return filename.split('_')[0]  # Extract the label from the filename

In [5]:
# empty features and labels
features = []
labels = []

images_data = []

# Iterate over each image
for img_path, img in tqdm(images):
    filename = os.path.basename(img_path)
    result_img, feature, minutiae_img = processing(img)
    label = extract_label(filename)  # Extract label from filename

    minutiae_info = feature.get('minutiae_list', [])
    singularities_info = feature.get('singularities_list', [])

    # Combine minutiae and singularity info into one list
    combined_info = minutiae_info + singularities_info

    # Create a dictionary to represent the image and its minutiae/singularities
    image_data = {
        'filename': filename,
        'label': label,
        'image': result_img,  # Store the processed image
        'features': combined_info  # Store the combined minutiae and singularities
    }

    # Append the image data dictionary to the list
    images_data.append(image_data)

    # Save the processed image
    cv.imwrite(os.path.join(output_dir, filename), result_img)
    cv.imwrite(os.path.join(minutiae_output_dir, filename), minutiae_img)

# Now, images_data list contains all the images along with their associated minutiae/singularities

# image pipeline
os.makedirs(output_dir, exist_ok=True)

100%|██████████| 88/88 [01:46<00:00,  1.21s/it]


In [36]:
# Load images and labels
image_dir = "./minutiae"
labels = []
images = []

for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        label = filename.split("_")[0]  # Extract label from filename
        image = cv2.imread(os.path.join(image_dir, filename), cv2.IMREAD_GRAYSCALE)
        images.append(image.flatten())  # Flatten image into a 1D array
        labels.append(label)

# Shuffle the data
combined = list(zip(images, labels))
random.shuffle(combined)
images[:], labels[:] = zip(*combined)


# Convert lists to numpy arrays
X = np.array(images)
y = np.array(labels)

print(X.shape)
print(y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


(88, 83824)
(88,)


In [6]:
# Train SVM classifier
svm_classifier = svm.SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

# Evaluate model
y_pred = svm_classifier.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy:", accuracy)

# Precision
precision = precision_score(y_test, y_pred, average='weighted')
print("SVM Precision:", precision)

# Recall
recall = recall_score(y_test, y_pred, average='weighted')
print("SVM Recall:", recall)

# F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')
print("SVM F1 Score:", f1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("SVM Confusion Matrix:")
print(conf_matrix)


SVM Accuracy: 0.5555555555555556
SVM Precision: 0.7777777777777778
SVM Recall: 0.5555555555555556
SVM F1 Score: 0.5925925925925926
SVM Confusion Matrix:
[[1 0 0 0 0 0 2 0 1 0 0]
 [0 2 0 0 1 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0]]


C:\Users\PEZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\PEZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
#random forest
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(X_train, y_train)

# Evaluate model
y_pred_rf = rf_classifier.predict(X_test)

# Accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

# Precision
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
print("Random Forest Precision:", precision_rf)

# Recall
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
print("Random Forest Recall:", recall_rf)

# F1 Score
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
print("Random Forest F1 Score:", f1_rf)

# Confusion Matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Random Forest Confusion Matrix:")
print(conf_matrix_rf)


Random Forest Accuracy: 0.5
Random Forest Precision: 0.4629629629629629
Random Forest Recall: 0.5
Random Forest F1 Score: 0.4333333333333333
Random Forest Confusion Matrix:
[[0 0 0 0 0 3 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1]]


C:\Users\PEZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\PEZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Reshape the training and testing data to 2D (flattening)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Initialize the k-NN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed

# Train the classifier
knn_classifier.fit(X_train_flat, y_train)

# Predict labels for the test data
y_pred_knn = knn_classifier.predict(X_test_flat)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy)


Accuracy: 0.3333333333333333


In [49]:
from collections import Counter

# Combine predictions using majority voting
hybrid_predictions = []
for rf_pred, knn_pred in zip(y_pred_rf, y_pred_knn):
    combined_pred = Counter([rf_pred, knn_pred]).most_common(1)[0][0]  # Choose the most common label
    hybrid_predictions.append(combined_pred)

# Calculate accuracy
hybrid_accuracy = accuracy_score(y_test, hybrid_predictions)
print("Hybrid Accuracy:", hybrid_accuracy)

Hybrid Accuracy: 0.2222222222222222


In [17]:
#MLPPPPPPPPPP
# Load images and labels
image_dir = "./minutiae"
labels = []
images = []

for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        label = filename.split("_")[0]  # Extract label from filename
        image = cv2.imread(os.path.join(image_dir, filename), cv2.IMREAD_GRAYSCALE)
        images.append(cv2.resize(image, (240, 320))) #for ratio purposes (3/4)
        labels.append(label)

# Shuffle the data
combined = list(zip(images, labels))
random.shuffle(combined)
images[:], labels[:] = zip(*combined)

# Convert lists to numpy arrays
X = np.array(images)
y = np.array(labels)

# Reshape the images to have a single channel (necessary for grayscale images)
X = X.reshape(X.shape[0], 320, 240, 1)

# Normalize the pixel values to the range [0, 1]
X = X.astype('float32') / 255.0

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the input shape
input_shape = X_train.shape[1:]

num_classes = 11

In [19]:
# Define the model architecture
inputs = Input(shape=input_shape)
x = layers.Flatten()(inputs)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(len(label_encoder.classes_), activation='softmax')(x)

# Create the model
model = models.Model(inputs, outputs)

# Define the Adam optimizer with adjusted hyperparameters
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# Compile the model with the Adam optimizer
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.0559 - loss: 49.3915 - val_accuracy: 0.0556 - val_loss: 184.2707
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.1118 - loss: 127.2491 - val_accuracy: 0.1111 - val_loss: 206.5822
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.0865 - loss: 161.2081 - val_accuracy: 0.1111 - val_loss: 203.9446
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.0812 - loss: 170.1731 - val_accuracy: 0.0556 - val_loss: 150.4264
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.1008 - loss: 112.6690 - val_accuracy: 0.1111 - val_loss: 104.9463
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.1001 - loss: 76.0021 - val_accuracy: 0.1111 - val_loss: 84.7947
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.0852 - loss: 58.3242 - val_accuracy: 0.0556 - val_loss: 80.7286
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.1294 - loss: 51.8513 - 

In [2]:
# Load images and labels
image_dir = "./minutiae"
labels = []
images = []

for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        label = filename.split("_")[0]  # Extract label from filename
        image = cv2.imread(os.path.join(image_dir, filename), cv2.IMREAD_GRAYSCALE)
        images.append(cv2.resize(image, (240, 320))) #for ratio purposes (3/4)
        labels.append(label)

# Shuffle the data
combined = list(zip(images, labels))
random.shuffle(combined)
images[:], labels[:] = zip(*combined)

# Convert lists to numpy arrays
X = np.array(images)
y = np.array(labels)

# Reshape the images to have a single channel (necessary for grayscale images)
X = X.reshape(X.shape[0], 320, 240, 1)

# Normalize the pixel values to the range [0, 1]
X = X.astype('float32') / 255.0

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the input shape
input_shape = X_train.shape[1:]

num_classes = 11

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN architecture
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(320, 240, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten layer to transition from convolutional to fully connected layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) # Adding dropout for regularization
model.add(Dense(11, activation='softmax')) # 11 classes for 11 fingers

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


C:\Users\PEZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 318, 238, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 159, 119, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 157, 117, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 78, 58, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 76, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 38, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 36, 26, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 18, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 11, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 5, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20480)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,621,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,190,987 (15.99 MB)

 Trainable params: 4,190,987 (15.99 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Predict labels for the test data
y_pred_nn = model.predict_classes(X_test)

# Accuracy
precision_nn = accuracy_score(y_test, y_pred_nn, average='weighted')
print("CNN Precision:", precision_nn)

# Precision
precision_nn = precision_score(y_test, y_pred_nn, average='weighted')
print("CNN Precision:", precision_nn)

# Recall
recall_nn = recall_score(y_test, y_pred_nn, average='weighted')
print("CNN Recall:", recall_nn)

# F1 Score
f1_nn = f1_score(y_test, y_pred_nn, average='weighted')
print("CNN F1 Score:", f1_nn)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 511ms/step - accuracy: 0.1132 - loss: 2.6954 - val_accuracy: 0.2222 - val_loss: 2.3743
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 465ms/step - accuracy: 0.0598 - loss: 2.4071 - val_accuracy: 0.0556 - val_loss: 2.3966
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 456ms/step - accuracy: 0.1275 - loss: 2.3967 - val_accuracy: 0.1667 - val_loss: 2.3934
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 422ms/step - accuracy: 0.0865 - loss: 2.4038 - val_accuracy: 0.1667 - val_loss: 2.3970
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 405ms/step - accuracy: 0.1489 - loss: 2.3987 - val_accuracy: 0.0556 - val_loss: 2.3969
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 415ms/step - accuracy: 0.0520 - loss: 2.3979 - val_accuracy: 0.0556 - val_loss: 2.3926
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 426ms/step - accuracy: 0.1385 - loss: 2.3837 - val_accuracy: 0.0000e+00 - val_loss: 2.3902
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 400ms/step - accuracy: 0.0592 - loss: 2.4048 - val_accuracy: 0.0000

AttributeError: 'Sequential' object has no attribute 'predict_classes'